In [7]:
#Import Dependencies
import pandas as pd
import time
from splinter import Browser
from bs4 import BeautifulSoup as bs
import requests

In [8]:
executable_path = {'executable_path': 'c:/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [9]:
#Mars news site scrape
url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
browser.visit(url)

#wait for page to load
time.sleep(2)

html = browser.html
soup = bs(html, 'html.parser')

#retrieve most recent news title and paragraph
news_title = soup.find('div', class_='content_title').text
news_p = soup.find('div', class_='article_teaser_body').text

print(news_title)
print(news_p)

Mars Now
In honor of Hispanic Heritage Month, Christina Hernandez, an instrument engineer on the Mars 2020 mission, talks about her childhood and journey to NASA.


In [10]:
#JPL Mars Image Connection
executable_path = {'executable_path': 'c:/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [11]:
#Mars image scrape
jpl_url = 'https://www.jpl.nasa.gov'
image_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(image_url)
html = browser.html
image_soup = bs(html, 'html.parser')

In [ ]:
#Feature Image Link
path = image_soup.find_all('img')[3]['src']
featured_image_url = jpl_url + path
print(featured_image_url)

In [12]:
#Mars Facts
facts_url = 'https://space-facts.com/mars/'
#get tables
facts_df = pd.read_html(facts_url)[0]
facts_df

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [13]:
#convert to an html table
facts_html = facts_df.to_html(index=False, header=False)
facts_html

'<table border="1" class="dataframe">\n  <tbody>\n    <tr>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <td>Surface Temperature:</td>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <td>First Record:</td>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <td>Recorded By:</td>\n      <td>Egyptian astronomers</td>\n    </tr>\n  </tbody>\n</table>'

In [14]:
#Mars Hemispheres
spheres_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(spheres_url)
spheres_html = browser.html
spheres_soup = bs(spheres_html, 'html.parser')


In [17]:
# Mars hemispheres products data
all_spheres = spheres_soup.find('div', class_='collapsible results')
mars_spheres = all_spheres.find_all('div', class_='item')

sphere_image_urls = []

#Iterate through each hemisphere data
for sphere in mars_spheres:
    # Collect Title
    hemisphere = sphere.find('div', class_="description")
    title = hemisphere.h3.text
    
    # Collect image link by browsing to hemisphere page
    hemisphere_link = hemisphere.a["href"]    
    browser.visit(spheres_url + hemisphere_link)
    
    time.sleep(2)
    
    image_html = browser.html
    image_soup = bs(image_html, 'html.parser')
    
    image_link = image_soup.find('div', class_='downloads')
    image_url = image_link.find('li').a['href']

    # Create Dictionary to store title and url info
    image_dict = {}
    image_dict['title'] = title
    image_dict['img_url'] = image_url
    
    hemisphere_image_urls.append(image_dict)

print(hemisphere_image_urls)

AttributeError: 'NoneType' object has no attribute 'find'